# Zero to Hero

## 1.08 Unos buenos hiperparametros

Se aplicará el modelo generado por rpart a los datos nuevos y se genera la salida para Kaggle

In [34]:
rm( list=ls())
gc()

library( "data.table")   #cargo la libreria  data.table
library( "rpart")  #cargo la libreria  rpart
library( "rpart.plot")
require("caret")

options(repr.plot.width=20, repr.plot.height=10) 
setwd("~/lab1/") #Aqui se debe poner la ruta de la PC local

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2369778,126.6,4511331,241.0,3058890,163.4
Vcells,4048076,30.9,147235574,1123.4,184018594,1404.0


In [54]:
set.seed(999953)
dataset <- fread("./datasets/competencia1_2022_v2.csv")   #cargo el dataset #cargo el dataset
datasettrain <- dataset[ foto_mes==202101 ]
train_rows <- createDataPartition(datasettrain$clase_ternaria, p= 0.66,list= FALSE)
dtrain <- datasettrain[c(train_rows)]
dapply <- datasettrain[-c(train_rows)]
nrow( dtrain )
ncol( dtrain )

[1] 106487

[1] 160

defino unos parametros interesantes
<br>
los hiperparametros van en una lista
<br>
notar la forma en que esos parametros se pasan a la funcion rpart

In [55]:
nrow( datasettrain )
nrow( dtrain )
nrow( dapply )

[1] 161342

[1] 106487

[1] 54855

In [56]:
param1  <- list("cp"= -0.937,
               "minsplit"=  1143,
               "minbucket"= 1,
               "maxdepth"= 9
              )

param2  <- list("cp"= 0.000001,
               "minsplit"=  18,
               "minbucket"= 3,
               "maxdepth"= 10
              )

In [57]:
#genero el modelo
modelo <-  rpart::rpart(formula= "clase_ternaria ~ ." ,
                        data= dtrain,
                        xval= 0,
                        control= param2)
print(modelo)

n= 106487 

node), split, n, loss, yval, (yprob)
      * denotes terminal node

   1) root 106487 1000 CONTINUA (0.0041225690 0.0052682487 0.9906091823)  
     2) ctrx_quarter< 13.5 5516  470 CONTINUA (0.0351704133 0.0500362582 0.9147933285)  
       4) status_targ_vs_creditos< 0.1 477  115 CONTINUA (0.0922431866 0.1488469602 0.7589098532)  
         8) mcuenta_corriente< -301.395 213   78 CONTINUA (0.1643192488 0.2018779343 0.6338028169)  
          16) mcomisiones< 169.27 87   46 CONTINUA (0.1954022989 0.3333333333 0.4712643678)  
            32) cliente_antiguedad< 64 40   21 BAJA+2 (0.2500000000 0.4750000000 0.2750000000)  
              64) cliente_edad>=53.5 6    0 BAJA+2 (0.0000000000 1.0000000000 0.0000000000) *
              65) cliente_edad< 53.5 34   21 BAJA+2 (0.2941176471 0.3823529412 0.3235294118)  
               130) mrentabilidad< -471.67 9    3 BAJA+1 (0.6666666667 0.0000000000 0.3333333333) *
               131) mrentabilidad>=-471.67 25   12 BAJA+2 (0.1600000000 0.5

In [58]:

#vim <- varImp(modelo,useModel=T)
#vim <- varImp(modelo)
#vim

summary(modelo)

Call:
rpart::rpart(formula = "clase_ternaria ~ .", data = dtrain, control = param2, 
    xval = 0)
  n= 106487 

             CP nsplit rel error
1  0.0016000000      0     1.000
2  0.0011428571     19     0.961
3  0.0010000000     26     0.953
4  0.0006666667     34     0.945
5  0.0005000000     40     0.941
6  0.0004444444     64     0.927
7  0.0003333333     76     0.921
8  0.0002500000     91     0.916
9  0.0001428571     95     0.915
10 0.0000010000    102     0.914

Variable importance
                      ctrx_quarter                       saldo_vs_mov 
                                 7                                  6 
           status_targ_vs_creditos                     mcuentas_saldo 
                                 6                                  6 
                     active_vs_mov                  mcuenta_corriente 
                                 6                                  5 
                   mactivos_margen               mrentabilidad_annual 
      

Aplico el modelo a los datos dapply  pidiendo que me devuelva probabildades

In [59]:
prediccion  <- predict( modelo, dapply, type = "prob")

Genero la salida

In [60]:
prob_baja2  <- prediccion[, "BAJA+2"]
Predicted   <- ifelse( prob_baja2 > 0.025, 1, 0 )  # 0.025 es la probabilidad de corte

entrega  <-  as.data.table( list( "numero_de_cliente"=dapply$numero_de_cliente, "Predicted"=Predicted)  )

#creo la carpeta donde va el experimento
dir.create( "./exp/",  showWarnings = FALSE ) 
dir.create( "./exp/ZH2018/", showWarnings = FALSE )

#genero el archivo para Kaggle
fwrite( entrega, 
        file= "./exp/ZH2018/para_Kaggle_0108_3.csv",
        sep=  "," )

¿Cuánto le da la ganancia en el Public Leaderboard?

In [61]:
preds <- predict(modelo, dapply, type = "class")
#preds

#confusionMatrix(preds,dtrain$clase_ternaria)
#table(preds)

#  BAJA+1   BAJA+2 CONTINUA 
#       4       14   161324 

confusionMatrix(as.factor(dapply$clase_ternaria), as.factor(preds), positive = "1")


Confusion Matrix and Statistics

          Reference
Prediction BAJA+1 BAJA+2 CONTINUA
  BAJA+1        8      6      211
  BAJA+2       10     13      266
  CONTINUA     39     69    54233

Overall Statistics
                                          
               Accuracy : 0.989           
                 95% CI : (0.9881, 0.9899)
    No Information Rate : 0.9974          
    P-Value [Acc > NIR] : 1               
                                          
                  Kappa : 0.0852          
                                          
 Mcnemar's Test P-Value : <2e-16          

Statistics by Class:

                     Class: BAJA+1 Class: BAJA+2 Class: CONTINUA
Sensitivity              0.1403509      0.147727         0.99128
Specificity              0.9960400      0.994960         0.25517
Pos Pred Value           0.0355556      0.044983         0.99801
Neg Pred Value           0.9991031      0.998626         0.07198
Prevalence               0.0010391      0.001604        

In [62]:
#nrow(dtrain)
#preds

In [63]:
#library(ROCR)
#preds  <- prediction(dapply$clase_ternaria), as.factor(preds))
#roc = performance(preds, measure="tpr", x.measure="fpr")
#plot(roc, col="orange", lwd=2) 
#lines(x=c(0, 1), y=c(0, 1), col="red", lwd=2)